In [1]:
!pip install scProject -U
import random
random.seed(a=613)
import numpy as np
import scProject
import scanpy as sc

dataset = sc.read('/home/abaraban/conda_env/humanRetinaData/humanRetinaDataset.h5ad')
dnorm = np.linalg.norm(dataset.X, axis=1, ord=1, keepdims=True)
dnormalized = dataset.X / (dnorm * .001)
dataset.X = dnormalized
patterns= sc.read('retina_patterns.h5ad')

scProject.matcher.orthologMapper(dataset, 'gene_map.csv', 'Gene stable ID','Mouse gene stable ID', 'id')
dataset_filtered, patterns_filtered = scProject.matcher.filterAnnDatas(dataset, patterns, 'id')

import matplotlib.pyplot as plt
from sklearn import linear_model
plt.rcParams['figure.figsize']= [10, 12]
dataset_filtered = scProject.matcher.logTransform(dataset_filtered)
scProject.rg.NNLR_ElasticNet(dataset_filtered, patterns_filtered, 'H01', .0001, .99, layer='log', iterations=100000)

Requirement already up-to-date: scProject in /home/abaraban/anaconda3/envs/scProjections/lib/python3.7/site-packages (1.0.9.8996)
(118555, 2412) dataset filter shape
(80, 2412) patterns filter shape


Trying to set attribute `.X` of view, copying.
Trying to set attribute `.X` of view, copying.


A layer named 'log' has been added to your filtered dataset
Regressing on loglayer of dataset_filtered
(118555, 80)


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt  

thresh = .5
outputs = dataset_filtered.obs['umap2_CellType']
classes = outputs.unique()
Predscores = np.zeros((80,len(classes)))
AUCScores = np.zeros((80,len(classes)))
source = []
target = []
weight = []
targetCellType = []
curves = {}
for i in range(80):
    model = RandomForestClassifier(class_weight = 'balanced') 
    pattern_weights = dataset_filtered.obsm['H01'][:,i]
    X_train, X_test, y_train, y_test = train_test_split(dataset_filtered.obsm['H01'][:,i].reshape(-1,1), dataset_filtered.obs['umap2_CellType'], test_size = 0.70, random_state=613, stratify=dataset_filtered.obs['umap2_CellType'] )
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    # print(predictions)
    for j in range(len(classes)):
        class_at_hand = model.classes_[j]
        indices = [] # index of all occurences of a class
        for idx in range(len(y_test)):
            if y_test[idx] == class_at_hand:
                indices.append(idx)
        # print(len(indices))
        score = 0
        for predict, truth in zip(predictions[indices], y_test[indices]):
            if predict == truth:
                score +=1
        score = score/len(indices)*100
        Predscores[i,j] = score
        # print(Predscores[i,j], i, class_at_hand, j)
    
    
"""    for j in range(len(classes)):
        class_at_hand = model.classes_[j]# classes[j]  # cell type to look at in this iteration
        # multiclass sklearn not implemented mucho annoying!!!
        binarize = {class_at_hand: 1}
        biny_test = [binarize.get(x, 0) for x in y_test]
        # bi_predictions = [binarize.get(x,0) for x in predictions]
        bi_predictions = model.predict_proba(X_test)[:, j]
        score = roc_auc_score(biny_test, bi_predictions)
        curves[(i, class_at_hand)] = roc_curve(biny_test, bi_predictions)
        if score > thresh:
            Predscores[i,j] = score
            source.append(i)
            target.append(j)
            targetCellType.append(class_at_hand)
            weight.append(score)"""




'    for j in range(len(classes)):\n        class_at_hand = model.classes_[j]# classes[j]  # cell type to look at in this iteration\n        # multiclass sklearn not implemented mucho annoying!!!\n        binarize = {class_at_hand: 1}\n        biny_test = [binarize.get(x, 0) for x in y_test]\n        # bi_predictions = [binarize.get(x,0) for x in predictions]\n        bi_predictions = model.predict_proba(X_test)[:, j]\n        score = roc_auc_score(biny_test, bi_predictions)\n        curves[(i, class_at_hand)] = roc_curve(biny_test, bi_predictions)\n        if score > thresh:\n            Predscores[i,j] = score\n            source.append(i)\n            target.append(j)\n            targetCellType.append(class_at_hand)\n            weight.append(score)'

In [3]:
import pandas as pd
humandf = pd.DataFrame(Predscores, columns=model.classes_, index=["Feature " + str(x+1) for x in range(80)])
humandf.to_csv('Highreghumanpredictionscores.csv')

In [4]:
humandf

,AC/HC_Precurs,Amacrine Cells,BC/Photo_Precurs,Bipolar Cells,Cones,Horizontal Cells,Muller Glia,Neurogenic Cells,RPCs,Retinal Ganglion Cells,Rods
Feature 1,0.000000,0.012863,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000
Feature 2,0.000000,0.257268,100.000000,1.463486,0.000000,0.145319,0.000000,0.000000,0.000000,0.000000,0.000000
Feature 3,0.751880,4.142012,0.896359,14.839748,1.634595,1.390907,11.602723,0.742574,10.286792,86.203536,4.775104
Feature 4,0.000000,0.000000,0.000000,0.043905,0.000000,0.000000,7.766089,0.000000,0.000000,99.972156,0.000000
Feature 5,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
Feature 76,0.075188,0.334448,0.000000,0.014635,0.000000,0.041520,100.000000,0.035361,1.169811,0.180983,0.094494
Feature 77,0.000000,0.553126,0.000000,6.863749,0.310366,0.062279,1.732673,0.035361,0.894340,99.749408,0.705556
Feature 78,0.000000,0.012863,0.056022,99.092639,0.103455,0.186838,12.778465,0.000000,13.671698,0.320200,0.163790
Feature 79,0.000000,0.000000,1.120448,4.112396,5.855576,0.000000,0.000000,99.752475,0.000000,0.000000,33.961194


In [5]:
mousedataset = sc.read('/home/abaraban/conda_env/mouseRetinaData/mouseRetinaDataset.h5ad')
varAdd = pd.read_csv('/home/abaraban/conda_env/mouseRetinaData/10x_Mouse_retina_pData_umap2_CellType_annot.csv', header=None)
mousedataset.obs['umap2_CellType'] = varAdd.drop(varAdd.index[[0]]).iloc[:,93].to_numpy()
mousepatterns= sc.read_h5ad('retina_patterns.h5ad')
mousedataset_filtered, mousepatterns_filtered = scProject.matcher.filterAnnDatas(mousedataset, mousepatterns, 'id')
mousedataset_filtered = scProject.matcher.logTransform(mousedataset_filtered)
scProject.rg.NNLR_ElasticNet(mousedataset_filtered, mousepatterns_filtered, 'mouse', .001, .99, layer='log')

thresh = .5
outputs = mousedataset_filtered.obs['umap2_CellType']
classes = outputs.unique()
mousescores = np.zeros((80,len(classes)))
mousePred = np.zeros((80, len(classes)))
source = []
target = []
weight = []
targetCellType = []
curves = {}
for i in range(80):
    model = RandomForestClassifier(class_weight='balanced') 
    pattern_weights = mousedataset_filtered.obsm['mouse'][:,i]
    X_train, X_test, y_train, y_test = train_test_split(mousedataset_filtered.obsm['mouse'][:,i].reshape(-1,1), mousedataset_filtered.obs['umap2_CellType'], test_size = 0.70, random_state=613, stratify=mousedataset_filtered.obs['umap2_CellType'])
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    for j in range(len(classes)):
        class_at_hand = model.classes_[j]
        indices = [] # index of all occurences of a class
        for idx in range(len(y_test)):
            if y_test[idx] == class_at_hand:
                indices.append(idx)
        # print(len(indices))
        score = 0
        for predict, truth in zip(predictions[indices], y_test[indices]):
            if predict == truth:
                score +=1
        score = score/len(indices)*100
        mousePred[i,j] = score
        # print(Predscores[i,j], i, class_at_hand, j)



/home/abaraban/anaconda3/envs/scProjections/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(107052, 3290) dataset filter shape
(80, 3290) patterns filter shape


Trying to set attribute `.X` of view, copying.
Trying to set attribute `.X` of view, copying.


A layer named 'log' has been added to your filtered dataset
Regressing on loglayer of dataset_filtered
(107052, 80)


In [6]:
import pandas as pd
mousedf = pd.DataFrame(mousePred, columns=model.classes_, index=["Feature " + str(x+1) for x in range(80)])
mousedf.to_csv('highregmouse_retina_predScores.csv')

In [7]:
mousedf

,Amacrine Cells,Bipolar Cells,Cones,Early RPCs,Horizontal Cells,Late RPCs,Muller Glia,Neurogenic Cells,Photoreceptor Precursors,Retinal Ganglion Cells,Rods
Feature 1,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Feature 2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000
Feature 3,0.710718,0.316331,0.566572,0.856454,0.088574,3.916296,1.711761,97.106463,0.314183,0.685255,3.255401
Feature 4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000
Feature 5,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
Feature 76,3.457549,0.000000,0.040469,0.523104,1.417183,0.105214,99.834346,3.019343,0.022442,11.767486,0.000000
Feature 77,2.612370,0.000000,0.080939,6.359300,0.000000,14.209726,0.000000,0.896367,0.246858,0.354442,99.861892
Feature 78,0.038417,99.960459,0.000000,27.155239,0.000000,18.786533,0.055218,0.141532,0.000000,0.000000,0.000000
Feature 79,0.000000,0.000000,1.335492,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009865
